In [1]:
import jax

from jax import random, jit, vmap, grad
from jax import numpy as jnp
from jax.lax import cond

import astropy
import numpy as np

import h5py
import astropy.units as u

from astropy.cosmology import Planck15, FlatLambdaCDM, z_at_value
import astropy.constants as constants
from jax.scipy.special import logsumexp
from scipy.interpolate import interp1d
from scipy.stats import gaussian_kde
from tqdm import tqdm

import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt
matplotlib.rcParams['font.family'] = 'Times New Roman'
matplotlib.rcParams['font.sans-serif'] = ['Bitstream Vera Sans']
matplotlib.rcParams['text.usetex'] = False
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['figure.figsize'] = (16.0, 10.0)
matplotlib.rcParams['axes.unicode_minus'] = False

import seaborn as sns
sns.set_context('talk')
sns.set_style('ticks')
sns.set_palette('colorblind')
c=sns.color_palette('colorblind')

jax.config.update("jax_enable_x64", True)
jax.config.update('jax_default_matmul_precision', 'highest')

In [2]:
from jaxinterp2d import interp2d, CartesianGrid

zMax_1 = 0.5
zMax_2 = 5

H0Planck = Planck15.H0.value
Om0Planck = Planck15.Om0

cosmo = FlatLambdaCDM(H0=H0Planck,Om0=Planck15.Om0)
speed_of_light = constants.c.to('km/s').value

zgrid_1 = np.expm1(np.linspace(np.log(1), np.log(zMax_1+1), 500))
zgrid_2 = np.expm1(np.linspace(np.log(zMax_1+1), np.log(zMax_2+1), 200))
zgrid = np.concatenate([zgrid_1,zgrid_2])

rs = []
Om0grid = jnp.linspace(Om0Planck-0.1,Om0Planck+0.1,50)
for Om0 in tqdm(Om0grid):
    cosmo = FlatLambdaCDM(H0=H0Planck,Om0=Om0)
    rs.append(cosmo.comoving_distance(zgrid).to(u.Mpc).value)

zgrid = jnp.array(zgrid)
rs = jnp.asarray(rs)
rs = rs.reshape(len(Om0grid),len(zgrid))

@jit
def E(z,Om0=Om0Planck):
    return jnp.sqrt(Om0*(1+z)**3 + (1.0-Om0))

@jit
def r_of_z(z,H0,Om0=Om0Planck):
    return interp2d(Om0,z,Om0grid,zgrid,rs)*(H0Planck/H0)

@jit
def dL_of_z(z,H0,Om0=Om0Planck):
    return (1+z)*r_of_z(z,H0,Om0)

@jit
def z_of_dL(dL,H0,Om0=Om0Planck):
    return jnp.interp(dL,dL_of_z(zgrid,H0,Om0),zgrid)

@jit
def dV_of_z(z,H0,Om0=Om0Planck):
    return speed_of_light*r_of_z(z,H0,Om0)**2/(H0*E(z,Om0))

@jit
def ddL_of_z(z,dL,H0,Om0=Om0Planck):
    return dL/(1+z) + speed_of_light*(1+z)/(H0*E(z,Om0))

In [ ]:
file = '../../../data/mocks_glass/mock_seed42_ratioNgalNagn100_fagn0.25.hdf5'

with h5py.File(file, 'r') as f:
    ras_ = np.asarray(f['ra_gal'])*np.pi/180
    decs_ = np.asarray(f['dec_gal'])*np.pi/180
    zs_ = np.asarray(f['z_gal'])
    
ngals = len(ras_)

In [ ]:
from pathos.multiprocessing import ProcessingPool, Pool
import tqdm_pathos

import healpy as hp

nside = 64
npix = hp.pixelfunc.nside2npix(nside)
apix = hp.pixelfunc.nside2pixarea(nside)
pixgrid = np.arange(npix)
print(pixgrid.shape)

ind = hp.pixelfunc.ang2pix(nside,np.pi/2-decs_,ras_)

def calculate_galaxies_pix(pix):
    cats = []
    ngalaxies = []
    id = np.where(ind == pix)[0]
    cats.append(id)
    ngalaxies.append(len(id))

    return cats, ngalaxies

p = ProcessingPool(20)
results = tqdm_pathos.map(calculate_galaxies_pix, list(pixgrid))

cats = []
ngalaxies = []

maxgals = 350
for pix in tqdm(pixgrid):
    gals = zs_[results[pix][0][0]]
    ngals = gals.shape[0]
    zgals = [gals]

    if ngals < maxgals:
        lenght = int(maxgals - ngals)
        zgals.append(100*np.ones(lenght))
    
    cats.append(np.concatenate(zgals))
    ngalaxies.append(ngals)
    
del results
p.close()

In [ ]:
m = np.asarray(ngalaxies)
hp.mollview(m, title="Mollview image RING")
hp.graticule()
plt.show()

In [ ]:
m.max()

In [ ]:
import h5py
with h5py.File('lognormal_pixelated_nside_'+str(nside)+'_galaxies.h5', 'w') as f:
    f.attrs['nside'] = nside
    f.create_dataset('zgals', data=np.asarray(cats), compression='gzip', shuffle=False)
    f.create_dataset('ngals', data=m, compression='gzip', shuffle=False)

In [ ]:
file = '../../../data/mocks_glass/mock_seed42_ratioNgalNagn100_fagn0.25.hdf5'

with h5py.File(file, 'r') as f:
    ras_ = np.asarray(f['ra_agn'])*np.pi/180
    decs_ = np.asarray(f['dec_agn'])*np.pi/180
    zs_ = np.asarray(f['z_agn'])
    
ngals = len(ras_)

In [ ]:
from pathos.multiprocessing import ProcessingPool, Pool
import tqdm_pathos

import healpy as hp

nside = 64
npix = hp.pixelfunc.nside2npix(nside)
apix = hp.pixelfunc.nside2pixarea(nside)
pixgrid = np.arange(npix)
print(pixgrid.shape)

ind = hp.pixelfunc.ang2pix(nside,np.pi/2-decs_,ras_)

def calculate_galaxies_pix(pix):
    cats = []
    ngalaxies = []
    id = np.where(ind == pix)[0]
    cats.append(id)
    ngalaxies.append(len(id))

    return cats, ngalaxies

p = ProcessingPool(20)
results = tqdm_pathos.map(calculate_galaxies_pix, list(pixgrid))

cats = []
ngalaxies = []

maxgals = 15
for pix in tqdm(pixgrid):
    gals = zs_[results[pix][0][0]]
    ngals = gals.shape[0]
    zgals = [gals]

    if ngals < maxgals:
        lenght = int(maxgals - ngals)
        zgals.append(100*np.ones(lenght))
    
    cats.append(np.concatenate(zgals))
    ngalaxies.append(ngals)
    
del results
p.close()

In [ ]:
m = np.asarray(ngalaxies)
hp.mollview(m, title="Mollview image RING")
hp.graticule()
plt.show()

In [ ]:
m.max()

In [ ]:
import h5py
with h5py.File('lognormal_pixelated_nside_'+str(nside)+'_agns.h5', 'w') as f:
    f.attrs['nside'] = nside
    f.create_dataset('zgals', data=np.asarray(cats), compression='gzip', shuffle=False)
    f.create_dataset('ngals', data=m, compression='gzip', shuffle=False)